In [ ]:
sentences = ['i love you', 'he loves me', 'she likes baseball', 'i hate you', 'sorry for that', 'this is awful']
labels = [1, 1, 1, 0, 0, 0] # 1 is good, 0 is not good

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

dtype = torch.FloatTensor

# Text-CNN Parameter
embedding_size = 2
sequence_length = 3
num_classes = 2
filter_sizes = [2, 2, 2]
num_filters = 3

In [ ]:
word_list = ' '.join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

inputs = []
for sen in sentences:
    inputs.append(np.array([word_dict[n] for n in sen.split()]))

targets = []
for out in labels:
    targets.append(out)

input_batch = Variable(torch.LongTensor(inputs))
target_batch = Variable(torch.LongTensor(targets))

In [ ]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()

        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Parameter(torch.empty(vocab_size, embedding_size).uniform_(-1, 1)).type(dtype)
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total, num_classes).uniform_(-1, 1)).type(dtype)
        self.Bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)

    def forward(self, X):
        embedded_chars = self.W[X]
        embedded_chars = embedded_chars.unsqueeze(1)

        pooled_outputs = []
        for filter_size in filter_sizes:

            conv = nn.Conv2d(1, num_filters, (filter_size, embedding_size), bias = True)(embedded_chars)      
            h = F.relu(conv)
            np = nn.MaxPool2d((sequence_length - filter_size + 1, 1))

            pooled = np(h).permute(0, 3, 2, 1)

        h_pool = torch.cat(pooled_outputs, -1)
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total])

        model = torch.nn(h_pool_flat, self.Weight) + self.Bias
        return model

model = TextCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

특정한 값으로 초기화를 하지 않는 행렬 생성

In [ ]:
torch.empty(3, 4)

## nn.MaxPool2d()

In [ ]:
import torch.__config__
m1 = nn.MaxPool2d(3, stride = 2)
print('m1:\n', m1, '\n')

m2 = nn.MaxPool2d((3, 2), stride = (2, 1))
input = torch.randn(20, 16, 50, 32)
output = m2(input)
print('m2:\n', m2, '\n')


In [ ]:
for epoch in range(5000):
    optimizer.zero_grad()
    output = model(input_batch)

    loss = criterion(output, target_batch)
    if(epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:6f'.format(loss))

    loss.backward()
    optimizer.step()

test_text = 'sorry hate you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = Variable(torch.LongTensor(tests))




# Pred
predict = model(test_batch).data.max(1, keepdin = True)[1]
if predict[0][0] == 0:
    print(test_text, 'is Bad Mean...')
else:
    print(test_text, 'is Good Mean!')